In [ ]:
import pandas as pd
import os

## Cilj:
Analizirati naslednje:
    -pri kateri starosti šahisti dosežejo svoj največji ELO
    -vpliv nacionalnosti na uspeh
    -ali lahko glede na performance nižjih 100 določimo kdo bo prišel v višjih 100

Zanima nas pri kateri starosti šahisti dosežejo svoj vrhunec. Postopek, da pridobimo to informacijo bo naslednji:
    1)Ustvarili bomo kopijo seznama sahisti ter v to kopijo shranili podatke o tem pri kateri starosti šahist doseže svoj največji ELO rating; podobno bomo naredili tudi z podatkom pri kateri starosti šahist doseže svoj največji indeks na lestvici najboljših 100 šahistov. Ta podatka nista nujno enakovredna, zaradi rating inflationa (https://en.wikipedia.org/wiki/Elo_rating_system#Mathematical_details)
    2)Statistično analizirali to - verjetno z preprosto aritmetično sredino.
    3) Kreacija grafikonov

In [76]:
pot = r'C:\Users\hugot\Documents\FMF\1_letnik\UVP-projektna-naloga\podatkovna_baza'
pot_shallow = r'C:\Users\hugot\Documents\FMF\1_letnik\UVP-projektna-naloga'
sahisti_elo = []

for filename in os.listdir(pot):
    ime_sahista = filename.replace('_', ' ')
    trenutni_maksimum = [(1-2000, 1)]
    datotecna_pot = os.path.join(pot, filename)
    podatki = pd.read_csv(datotecna_pot, index_col = 'datum', sep = ',')
    for datum, vrstica in podatki.iterrows():
        rating = int(vrstica['rating'])
        if rating > trenutni_maksimum[0][1]:
            trenutni_maksimum = [(datum, rating)]
        #naslednja vrstica poskrbi, da ne zgrešimo, če šahist doseže svoj najvišji 
        #rating več kot enkrat.
        elif rating == trenutni_maksimum[0][1]: 
            trenutni_maksimum.append((datum, rating))
    sahisti_elo.append({'Ime': ime_sahista, 'Maksimumi' : trenutni_maksimum})
#sahisti_elo je seznam slovarjev, vsebujoc informacje o imenu ter
#maksimalnih ratingih (datum, rating) kjer je rating sahistov najvisji rating, 
#datum pa eden izmed datumov, na katerega je slednjega dosegel

#Sedaj bomo v seznam slovarjev sahisti_elo za vsakega šahista dodali še starost, pri kateri je dosegel maksimalni rating, če je teh več pa več starosti.
general_info = pd.read_csv(os.path.join(pot_shallow, 'sahisti'), sep=',')

print(type(general_info))

for sahist in sahisti_elo:
    ime_sahista = sahist['Ime']
    maksimumi = sahist['Maksimumi']
    sahist['Najvišji rating'] = maksimumi[0][1]
    #Opomnik: maksimum je seznam nizov dveh podatkov, prvi je datum, drugi pa maksimalni elo šahista
    for indeks, vrstica in general_info.iterrows():
        if vrstica['Ime'] == ime_sahista:
            sahist['Leto rojstva'] = vrstica['Leto rojstva']
    for maksimum in maksimumi:
        sahist['Starosti ob najvišjih ratingih'] = [int(maksimum[1]-sahist['Leto rojstva'])]

sahisti_elo_pandas = pd.DataFrame(sahisti_elo, columns=['Ime', 'Leto rojstva', 'Maksimumi'])

sahisti_elo_pandas.head(10)

<class 'pandas.core.frame.DataFrame'>


,Ime,Leto rojstva,Maksimumi
0,"Abasov, Nijat",1995,"[(11-2023, 2679), (10-2023, 2679)]"
1,"Abdusattorov, Nodirbek",2004,"[(07-2024, 2769)]"
2,"Acs, Peter",1981,"[(01-2003, 2623)]"
3,"Adams, Michael",1971,"[(04-2017, 2761), (03-2017, 2761), (09-2013, 2..."
4,"Adhiban, B",1992,"[(05-2019, 2701), (04-2019, 2701)]"
5,"Adianto, Utut",1965,"[(07-2001, 2598)]"
6,"Afromeev, Vladimir",1954,"[(07-2008, 2646), (04-2008, 2646), (01-2008, 2..."
7,"Agrest, Evgenij",1966,"[(01-2004, 2616)]"
8,"Akobian, Varuzhan",1983,"[(06-2017, 2673)]"
9,"Akopian, Vladimir",1971,"[(10-2007, 2713), (10-2006, 2713), (07-2006, 2..."
